In [ ]:
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
from datasets import Dataset
import mlflow
import torch
from transformers import Trainer, TrainingArguments
from transformer_lens import utils, HookedTransformer
from transformers import (
    GPT2Tokenizer, 
    GPT2LMHeadModel, 
    DataCollatorForLanguageModeling, 
    Trainer, 
    TrainingArguments
)
import mlflow
from transformers.integrations import MLflowCallback

from create_dataset import make_rows_from_chat

file_name = "artifacts/input_text.txt"

with open(file_name, "r") as file_read:
    chat = file_read.read()

chat_rows = make_rows_from_chat(chat)